In [15]:
admissions = pd.read_csv("admissions.csv", on_bad_lines='skip')  # pandas ≥ 1.3


In [17]:
with open("admissions.csv") as f:
    lines = f.readlines()
    print(lines[28694:28700])  # 0-индексация


['10550646,21218420,2124-11-07 00:32:00,2124-11-11 13:46:00,,OBSERVATION ADMIT,P045WF,EMERGENCY ROOM,HOME,00,EW EMER.,P71XWZ,EMERGENCY ROOM,DIED,Medicare,English,WIDOWED2,Engle,English,WIDOWED2M93QMB,PHYSICIAN R293QMB0NCY ROOM,HOME,00,EWEnglROOM,DIED,Medi,ELECT4.,P21DYB,WA700,24067e,Engli-04-2450:00,LK-IN/SE-04-2450SICIAN REDURE SITE,,Medicare,E0YD972-06-301te,Engli6,English,WDIRECT EM021glish,WDIRECT EM021gl,2112-04-06 15:59:00,2112-04-06 17:15:00,0\n', '7,DIED,Med5:59:00,ED,BLAALTH CARE,M3-IN/SELF4BSERVATI125-07-07 08:34:00,0\n', '10550646e,EnglINGLE,WHITE,2168-09-03 17:44:00,2168-09-03 18:31:00,0IED,WHITE,,,0\n', '1055IAN REF150641,22663e,English,WIDME,M80,2English6-10-E-04-245A3:37:001OYKILLED NURSING FACILITY,SKILLED NURSING FACED,WHITE,,,0\n', '1055IAN REF150L,HO4259852AN REF2W8,PHYSICIAN2AN RE-28 1735SXX,PH-12 11:4583134-01-07 15:31:00,0\n', '10549659,20114661,2123-12-08 00:53:00,2123-122 21:21:0141-06-70,206h6-10-,2146-0N2AN RE-2810-,212Y ROOM,SKIL 08:34:00,0\n']


In [18]:
import pandas as pd

# Прочитать admissions.csv, пропуская строки с ошибками
admissions = pd.read_csv("admissions.csv", on_bad_lines='skip')

# Пересохранить "чистую" версию
admissions.to_csv("admissions_clean.csv", index=False)

print("✅ admissions_clean.csv создан без битых строк.")


✅ admissions_clean.csv создан без битых строк.


In [4]:
import pandas as pd
import numpy as np
import os
from mlxtend.frequent_patterns import fpgrowth, association_rules
patients = pd.read_csv('patients.csv')
admissions = pd.read_csv('admissions.csv')
diagnoses = pd.read_csv('diagnoses_icd.csv')
d_icd_diagnoses = pd.read_csv('d_icd_diagnoses.csv')
d_icd_procedures = pd.read_csv('d_icd_procedures.csv')
procedures = pd.read_csv('procedures_icd.csv')
symptoms = pd.read_csv('Notes.csv')

admissions['hadm_id'] = pd.to_numeric(admissions['hadm_id'], errors='coerce')
admissions['subject_id'] = pd.to_numeric(admissions['subject_id'], errors='coerce')
admissions.dropna(subset=['hadm_id', 'subject_id'], inplace=True)
admissions = admissions.astype({'hadm_id':'int32', 'subject_id':'int32'})

sample_fraction = 0.01
admissions = admissions.sample(frac=sample_fraction, random_state=42)
sampled_subject_ids = set(admissions['subject_id'])
sampled_hadm_ids = set(admissions['hadm_id'])


patients = patients[patients['subject_id'].isin(sampled_subject_ids)]
diagnoses = diagnoses[diagnoses['hadm_id'].isin(sampled_hadm_ids)]
procedures = procedures[procedures['hadm_id'].isin(sampled_hadm_ids)]
symptoms = symptoms[symptoms['hadm_id'].isin(sampled_hadm_ids)]

diagnoses_desc = diagnoses.merge(d_icd_diagnoses, on=['icd_code', 'icd_version'], how='left')
diagnoses_desc['long_title'].fillna('Unknown_Diagnosis', inplace=True)

procedures_desc = procedures.merge(d_icd_procedures, on=['icd_code', 'icd_version'], how='left')
procedures_desc['long_title'].fillna('Unknown_Procedure', inplace=True)

transactions_base = admissions[['hadm_id', 'subject_id']].merge(
    patients[['subject_id', 'anchor_age', 'gender']], on='subject_id', how='left')

primary_diag = diagnoses_desc[diagnoses_desc['seq_num'] == 1][['hadm_id', 'long_title']]
transactions_base = transactions_base.merge(primary_diag, on='hadm_id', how='left').rename(columns={'long_title': 'primary_diagnosis'})

symptoms = symptoms[['hadm_id', 'Symptoms']].fillna('No_Symptom')
transactions_base = transactions_base.merge(symptoms, on='hadm_id', how='left').fillna({'Symptoms':'No_Symptom'})

#ranging symptoms
top_symptoms = transactions_base['Symptoms'].value_counts().nlargest(50).index
transactions_base['Symptoms'] = transactions_base['Symptoms'].where(transactions_base['Symptoms'].isin(top_symptoms), 'Other')

top_procedures = procedures_desc['long_title'].value_counts().nlargest(100).index
procedures_desc['long_title'] = procedures_desc['long_title'].where(procedures_desc['long_title'].isin(top_procedures), 'Other_Procedure')

top_diagnoses = transactions_base['primary_diagnosis'].value_counts().nlargest(100).index
transactions_base['primary_diagnosis'] = transactions_base['primary_diagnosis'].where(transactions_base['primary_diagnosis'].isin(top_diagnoses), 'Other_Diagnosis')

def memory_efficient_dummies(df, col, prefix):
    return pd.get_dummies(df[col], prefix=prefix, dtype='uint8')

demographic_dummies = memory_efficient_dummies(transactions_base, 'gender', 'Gender')
symptom_dummies = memory_efficient_dummies(transactions_base, 'Symptoms', 'Symptom')
diagnosis_dummies = memory_efficient_dummies(transactions_base, 'primary_diagnosis', 'Diagnosis')

procedure_dummies = pd.get_dummies(procedures_desc[['hadm_id', 'long_title']],
                                   columns=['long_title'], prefix='Procedure', dtype='uint8').groupby('hadm_id').max()
transactions_matrix = pd.concat([
    demographic_dummies.set_index(transactions_base['hadm_id']),
    symptom_dummies.set_index(transactions_base['hadm_id']),
    procedure_dummies,
    diagnosis_dummies.set_index(transactions_base['hadm_id'])
], axis=1).fillna(0).astype('uint8')

os.makedirs('output', exist_ok=True)
transactions_matrix.to_csv('output/transactions_matrix.csv')
detailed_matrix = transactions_base[['hadm_id', 'subject_id', 'anchor_age', 'gender', 'primary_diagnosis', 'Symptoms']].copy()
procedure_mapping = procedures_desc.groupby('hadm_id')['long_title'].apply(list).to_dict()
detailed_matrix['Procedures'] = detailed_matrix['hadm_id'].map(procedure_mapping)
detailed_matrix.to_csv('output/detailed_transaction_matrix.csv', index=False)
frequent_itemsets = fpgrowth(transactions_matrix, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.5)
rules.to_csv('output/association_rules.csv', index=False)


<ipython-input-4-b905d6dfd1fa>:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  diagnoses_desc['long_title'].fillna('Unknown_Diagnosis', inplace=True)
<ipython-input-4-b905d6dfd1fa>:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

In [5]:
#filtering
diagnosis_cols = [col for col in transactions_matrix.columns if col.startswith('Diagnosis_')]

rules['consequents'] = rules['consequents'].apply(lambda x: set(x))
rules['antecedents'] = rules['antecedents'].apply(lambda x: set(x))

filtered_rules = rules[rules['consequents'].apply(lambda cons: all(c in diagnosis_cols for c in cons))]

filtered_rules.to_csv('output/diagnosis_consequents_rules.csv', index=False)



✅ Отфильтровано правил с диагнозами в consequents: 36
📁 Результаты сохранены в 'output/diagnosis_consequents_rules.csv'.
